In [60]:
from __future__ import print_function
%matplotlib inline
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir
from os.path import isfile, join
from numpy import median, diff

# Helpers to read .sm and return notes and meta data
- get_notes_from_note_string(note_string)
- get_notes_and_metadata(file)
- get_song_steps()

In [61]:
regex_notes_with_metadata = '#NOTES:n     dance-single((?:(?!//-).)*);'
regex_metadata_split = ':n     (.*):n     (.*):n     (.*):n     (.*):n     (.*):(.*);'
def get_notes_from_note_string(note_string):
    note_strings_split = re.split(r'n', note_string)[1:-1]
    notes = []
    bar = []
    for row in note_strings_split:
        if len(row) == 4:
            bar.append(row)
        else:
            notes.append(bar)
            bar = []
    return notes

def get_notes_and_metadata(file):
    difficulty_map = {}
    with open(file) as txt:
        step_file = txt.read()
        step_file = step_file.replace('\n', 'n')
        notes_with_metadata_groups = re.finditer(regex_notes_with_metadata, step_file)
        for match in notes_with_metadata_groups:
            notes_with_metadata = match.group(0)
            split_data = re.search(regex_metadata_split, notes_with_metadata)
            difficulty = split_data.group(4)
            metadata = split_data.group(5)
            notes = get_notes_from_note_string(split_data.group(6))
            notes_with_metadata_map = {
                'DIFFICULTY': difficulty,
                'METADATA': metadata,
                'NOTES': notes,
            }
            difficulty_map[difficulty] = notes_with_metadata_map
    return difficulty_map

# Notes Structure
- Normal notes are 1

1000
- Holds start 2 end 3

2000
0000
0000
3000
- Rolls start 4 end 3

4000
0000
0000
3000
- Mines are M

MMMM

In [95]:
notes_per_bar = 48
def padBar(bar):
    pad = int(48 / len(bar)) if len(bar) != 0 else 1
    return [row for note in bar for row in [note] + (pad - 1) * ['0000']]

def get_plain_padded_notes_from_note_string(pack, song):
    folder = 'StepMania/Songs/{0}/{1}/'.format(pack, song)
    stepfile = folder + next(file for file in listdir(folder) if file.endswith('.ssc') or file.endswith('.sm'))
    notes_and_metadata = get_notes_and_metadata(stepfile)
    notes_for_difficulty = min(notes_and_metadata.values(), key=lambda steps:abs(int(steps['DIFFICULTY']) - 9))['NOTES']
    return [row for bar in notes_for_difficulty for row in padBar(bar)]

In [97]:
packs = ['In The Groove', 'In The Groove 2', 'In The Groove 3', 'In The Groove Rebirth', 'In The Groove Rebirth +', 'In The Groove Rebirth 2 (BETA)', 'Piece of Cake', 'Piece of Cake 2', 'Piece of Cake 3', 'Piece of Cake 4', 'Piece of Cake 5']
packs = ['Piece of Cake', 'Piece of Cake 2', 'Piece of Cake 3', 'Piece of Cake 4', 'Piece of Cake 5']

for pack in packs:
    songs = [song for song in listdir('StepMania/Songs/{0}'.format(pack)) if song != '.DS_Store']
    for song in songs:
        if '{0}~{1}_notes.csv'.format(pack, song) in listdir('data'):
            print ('Song Already Loaded')
        else:
            notes = get_plain_padded_notes_from_note_string(pack, song)
            pd.DataFrame(notes).to_csv('data/{0}~{1}_notes.csv'.format(pack, song), index=False)

Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
